# Interp on my GPT-2 Flax Implementation

In [1]:
# Imports
import jax
import jax.numpy as jnp
from jax import random
from model import GPT, GPTConfig
import os

import penzai
from __future__ import annotations

import os
import gc

import jax
import jax.numpy as jnp
import numpy as np
import orbax.checkpoint
import optax
from jax.experimental import mesh_utils

import treescope
import penzai
from penzai import pz

import sentencepiece as spm

from penzai.models import transformer
from penzai.models import simple_mlp, transformer

from penzai.toolshed import token_visualization
from penzai.toolshed import basic_training
from penzai.toolshed import jit_wrapper
from penzai.nn.layer import Layer
import dataclasses
treescope.basic_interactive_setup(autovisualize_arrays=True)


## Load model from checkpoint

In [2]:
from flax.training import checkpoints

config = GPTConfig(
    vocab_size=50257,  # Updated for GPT-2 BPE
    block_size=256,  # or whichever block size you used
    n_layer=4,
    n_head=4,
    n_embd=256,
    embd_pdrop=0.0,  # 0 for inference
    resid_pdrop=0.0,
    attn_pdrop=0.0,
)

# Load your model from a checkpoint directory
ckpt_dir = os.path.abspath("checkpoints")

model = GPT(config)
params = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=None)
if params is None:
    raise ValueError(f"No checkpoint found in {ckpt_dir}")


/home/j/anaconda3/envs/mechinterp/lib/python3.11/site-packages/orbax/checkpoint/_src/serialization/type_handlers.py:1175: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


### Count number of parameters

In [3]:

# Create a dummy input. The shape should match the model's expected input shape.
# Here we use a batch size of 1 and sequence length equal to block_size.
dummy_input = jnp.zeros((1, config.block_size), dtype=jnp.int32)

# Initialize the model parameters with a PRNG key
key = random.PRNGKey(0)

# Function to count total parameters
def count_params(params):
    # Use jax.tree_util.tree_leaves to get all arrays in the nested dict
    return sum(x.size for x in jax.tree_util.tree_leaves(params))


total_params = count_params(params)
print("Total number of parameters:", total_params)

Total number of parameters: 28923904


## Generating completions via Inference

We can generate the completion in one big block...

In [4]:
from inference import generate_text
from bpe import BPETokenizerJax

# Example prompt
prompt = "Shakespeare wrote:"

tokenizer = BPETokenizerJax()

# Generate
completion = generate_text(
    prompt=prompt,
    tokenizer=tokenizer,
    model=model,
    params=params,
    max_new_tokens=5,
    temperature=0.7,
    top_k=10,
)[len(prompt) :]

print(f"Prompt: {prompt}")
print("Generated text:")
print(completion)

Generating tokens: 100%|██████████| 5/5 [00:06<00:00,  1.34s/it]

Prompt: Shakespeare wrote:
Generated text:
 , but the story was


Or yield the completion token-by-token...

In [5]:
from inference import generate_text
from bpe import BPETokenizerJax

# Example prompt
prompt = "Shakespeare wrote:"
prompt = "You are gentlemen of brave metal; you would lift the moon out of her sphere, if she would continue in it five weeks without changing."
prompt = prompt +  prompt
tokenizer = BPETokenizerJax()

# Tokenize the prompt via BPE
# The tokenizer(...) call returns a list of jnp arrays (one per input string).
encoded_list = tokenizer(prompt, return_tensors="jax")
# For a single prompt string, encoded_list is a list of length 1.
idx = encoded_list[0]  # shape (sequence_length,)

# Add a batch dimension (shape: (1, seq_len))
idx_jax = idx[jnp.newaxis, :]

# Initialize RNG key
rng = jax.random.PRNGKey(0)

print("Prompt:", prompt)

print("Generated text:")

# Generate text
for new_token in model.apply(
        {"params": params},
        idx_jax,
        max_new_tokens=10,
        rng=rng,
        temperature=0.7,
        do_sample=True,
        top_k=None,
        method=model.generate_yield,
    ):
    # Decode the generated token
    generated_text = tokenizer.decode(new_token)
    print(generated_text, end="")



Prompt: You are gentlemen of brave metal; you would lift the moon out of her sphere, if she would continue in it five weeks without changing.You are gentlemen of brave metal; you would lift the moon out of her sphere, if she would continue in it five weeks without changing.
Generated text:
 to the same era . It

E0210 00:07:24.751517   33792 hlo_lexer.cc:443] Failed to parse int literal: 6818125836963382500036


 will not be a

Or write an external function which generates that completion token-by-token...

In [6]:
import jax
import jax.numpy as jnp
import flax.linen as nn

def external_generate(
    model: nn.Module,
    params: dict,
    idx: jnp.ndarray,
    max_new_tokens: int,
    rng: jax.random.PRNGKey,
    temperature: float = 1.0,
    do_sample: bool = False,
    top_k: int = None,
):
    """
    Generate tokens from the model using only the __call__ method.

    Args:
        model: The GPT model (an instance of nn.Module) with a __call__ method.
        params: The parameters of the model.
        idx: jnp.ndarray of shape (B, T) containing the initial token indices.
        max_new_tokens: Number of tokens to generate.
        rng: A JAX random key.
        temperature: Temperature factor for sampling.
        do_sample: If True, sample from the distribution; otherwise, take argmax.
        top_k: If provided, restrict sampling to the top_k logits.

    Yields:
        Generated tokens one at a time (each of shape (1,)).
    """
    for _ in range(max_new_tokens):
        # Crop the context to the model's block size if necessary
        if idx.shape[1] > model.config.block_size:
            idx_cond = idx[:, -model.config.block_size:]
        else:
            idx_cond = idx

        # FIX: Wrap the parameters in {"params": params} so that the "params" collection is set correctly.
        logits, _ = model.apply({"params": params}, idx_cond, deterministic=True)

        # Focus on the logits for the last token in the sequence and scale by temperature
        logits = logits[:, -1, :] / temperature

        # Optionally, apply top-k filtering
        if top_k is not None:
            # Get the top_k logits and determine the minimum value among them
            top_logits, _ = jax.lax.top_k(logits, top_k)
            k_threshold = jnp.min(top_logits, axis=-1, keepdims=True)
            # Replace logits not in the top_k with -infinity
            logits = jnp.where(logits < k_threshold, -jnp.inf, logits)

        # Convert logits to probabilities
        probs = nn.softmax(logits, axis=-1)

        # Sample the next token (or take the argmax)
        rng, subkey = jax.random.split(rng)
        if do_sample:
            next_token = jax.random.categorical(subkey, jnp.log(probs), axis=-1)
        else:
            next_token = jnp.argmax(probs, axis=-1)

        # Ensure the next token has shape (B, 1) and append it to the current sequence
        next_token = next_token[:, None]
        idx = jnp.concatenate([idx, next_token], axis=1)
        yield next_token[0]

# Example usage:
print("Prompt:", prompt)
print("Generated text:")

for new_token in external_generate(
        model=model,
        params=params,  # This should be the inner parameters; external_generate now wraps them correctly.
        idx=idx_jax,
        max_new_tokens=20,
        rng=rng,
        temperature=0.8,
    ):
    # Decode the generated token
    generated_text = tokenizer.decode(new_token)
    print(generated_text, end="")


Prompt: You are gentlemen of brave metal; you would lift the moon out of her sphere, if she would continue in it five weeks without changing.You are gentlemen of brave metal; you would lift the moon out of her sphere, if she would continue in it five weeks without changing.
Generated text:
 . 
 
 = = = = = 
 
 = = = = = =

## Unflaxify the Model for Penzai

In [7]:
from penzai.toolshed.unflaxify import unflaxify_apply

# Create dummy input
dummy_input = jnp.ones((1, config.block_size), dtype=jnp.int32)
rng = jax.random.PRNGKey(42)
init_rng, dropout_rng = jax.random.split(rng)

# Initialize
variables = model.init(
    {
        "params": init_rng,
        "dropout": dropout_rng
    },
    idx=dummy_input,
    deterministic=False
)
    
# Intercept the forward pass
pz_model = unflaxify_apply(
    module=model,
    variables=variables,
    idx=dummy_input,     # our forward arguments
    deterministic=True   # example kwarg
)

pz_model

InterceptedFlaxModuleMethod(
  module=GPT(
    config=<model.GPTConfig object at 0x7fd1fe36d110>, # Repeated python obj at 0x7fd1fe36d110
    name=None,
  ),
  method_name='__call__',
  scope_data=None,
  submodule_calls={(0, 'wte.__call__'): InterceptedFlaxModuleMethod(module=Embed(num_embeddings=50257, features=256, dtype=None, param_dtype=jax.numpy.float32, embedding_init=<function normal.<locals>.init at 0x7fd1f9da9b20>, name='wte'), method_name='__call__', scope_data=InterceptedFlaxScopeData(parameters={'embedding': Parameter(label='wte.embedding', value=<jax.Array float32(50257, 256)- too large to summarize.>, metadata={})}, variables={}, immutable_variables={}, rng_names=frozenset({})), submodule_calls={}), (1, 'wpe.__call__'): InterceptedFlaxModuleMethod(module=Embed(num_embeddings=128, features=256, dtype=None, param_dtype=jax.numpy.float32, embedding_init=<function normal.<locals>.init at 0x7fd1f9da8180>, name='wpe'), method_name='__call__', scope_data=InterceptedFlaxScopeData(parameters={'embedding': Parameter(label='wpe.embedding', value=<jax.Array float32(128, 256) ≈-0.0001 ±0.02 [≥-0.083, ≤0.08] nonzero:32_768>, metadata={})}, variables={}, immutable_variables={}, rng_names=frozenset({})), submodule_calls={}), (2, 'drop.__call__'): InterceptedFlaxModuleMethod(module=Dropout(rate=0.0, broadcast_dims=(), deterministic=None, rng_collection='dropout', name='drop'), method_name='__call__', scope_data=None, submodule_calls={}), (3, 'h_0.__call__'): InterceptedFlaxModuleMethod(module=Block(config=<model.GPTConfig object at 0x7fd1fe36d110>, name='h_0'), method_name='__call__', scope_data=None, submodule_calls={(0, 'ln_1.__call__'): InterceptedFlaxModuleMethod(module=LayerNorm(epsilon=1e-05, dtype=None, param_dtype=jax.numpy.float32, use_bias=True, use_scale=True, bias_init=zeros, scale_init=ones, reduction_axes=-1, feature_axes=-1, axis_name=None, axis_index_groups=None, use_fast_variance=True, force_float32_reductions=True, name='ln_1'), method_name='__call__', scope_data=InterceptedFlaxScopeData(parameters={'bias': Parameter(label='h_0.ln_1.bias', value=<jax.Array float32(256,) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:256>, metadata={}), 'scale': Parameter(label='h_0.ln_1.scale', value=<jax.Array float32(256,) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:256>, metadata={})}, variables={}, immutable_variables={}, rng_names=frozenset({})), submodule_calls={}), (1, 'attn.__call__'): InterceptedFlaxModuleMethod(module=CausalSelfAttention(config=<model.GPTConfig object at 0x7fd1fe36d110>, name='attn'), method_name='__call__', scope_data=None, submodule_calls={(0, 'c_attn.__call__'): InterceptedFlaxModuleMethod(module=Dense(features=768, use_bias=True, dtype=None, param_dtype=jax.numpy.float32, precision=None, kernel_init=<function normal.<locals>.init at 0x7fd1f9b90360>, bias_init=zeros, dot_general=None, dot_general_cls=None, name='c_attn'), method_name='__call__', scope_data=InterceptedFlaxScopeData(parameters={'bias': Parameter(label='h_0.attn.c_attn.bias', value=<jax.Array float32(768,) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:768>, metadata={}), 'kernel': Parameter(label='h_0.attn.c_attn.kernel', value=<jax.Array float32(256, 768) ≈3.9e-06 ±0.02 [≥-0.091, ≤0.1] nonzero:196_608>, metadata={})}, variables={}, immutable_variables={}, rng_names=frozenset({})), submodule_calls={}), (1, 'attn_dropout.__call__'): InterceptedFlaxModuleMethod(module=Dropout(rate=0.0, broadcast_dims=(), deterministic=None, rng_collection='dropout', name='attn_dropout'), method_name='__call__', scope_data=None, submodule_calls={}), (2, 'c_proj.__call__'): InterceptedFlaxModuleMethod(module=Dense(features=256, use_bias=True, dtype=None, param_dtype=jax.numpy.float32, precision=None, kernel_init=<function normal.<locals>.init at 0x7fd1f9b90cc0>, bias_init=zeros, dot_general=None, dot_general_cls=None, name='c_proj'), method_name='__call__', scope_data=InterceptedFlaxScopeData(parameters={'bias': Parameter(label='h_0.attn.c_proj.bias', value=<jax.Array float32(256,) ≈0.0 ±0.0 [≥0.0, ≤0.0]

# Induction Heads

We start by creating a repeated sequence...

In [8]:

repeated_sequence_prompt = "You are gentlemen of brave metal; you would lift the moon out of her sphere, if she would continue in it five weeks without changing."

# repeat the prompt twice
repeated_sequence = repeated_sequence_prompt + repeated_sequence_prompt

# Tokenize the prompt via BPE
tokens = tokenizer(repeated_sequence)

tokens[0]



<jax.Array int32(56,) [≥11, ≤28_527] nonzero:56
  <Arrayviz rendering>
| Device: GPU 0>

In [9]:
from penzai.toolshed.unflaxify import ArgsAndKwargs

# Wrap tokens and tag with "batch" and "seq"
token_seq = pz.nx.wrap(tokens).tag("batch", "seq")

# Remove the tags and unwrap to get a plain JAX array
token_seq_plain = token_seq.untag("batch", "seq").unwrap()

# Now capture the plain array inside an ArgsAndKwargs instance.
token_seq_wrapped = ArgsAndKwargs.capture(token_seq_plain)

# Pass the wrapped, plain array to the model.
logits = pz_model(token_seq_wrapped)[0][0] # [0] removes the batch dim 

logits

# Find the index of the highest logit
highest_logit_index = jnp.argmax(logits, axis=-1)
print(highest_logit_index)

[27388 29435  6547 47284 32172 19143 11544  2259 31272 18919 46963 19059
 33337 24560 33337  4444 49866 21990 26508  2208 37767 14302 35522 23020
 33337 38316 30566 38776 19059 32239 21292 21292 33337 33337 33154 23210
 23210 35522 35522 28449 11894 32244 12272 38330 14302 45349 28449 36390
 28449 38776 35522 29296 32101 38316 12272   525]


In [10]:
logits =pz.nx.wrap(logits).tag("seq", "vocabulary")


# Map softmax over the vocabulary
log_probs = pz.nx.nmap(jax.nn.log_softmax)(
    logits.untag("vocabulary")
).tag("vocabulary")
log_probs


<NamedArray float32(| seq:56, vocabulary:50257) ≈-1.1e+01 ±0.32 [≥-1.3e+01, ≤-9.4] nonzero:2_814_392 (wrapping jax.Array)
  <Arrayviz rendering>
| Device: GPU 0>

In [11]:
# Indexing with a dictionary indexes the named axes; pz.slice helps slice them.
sliced_preds = log_probs[{"seq": pz.slice[:-1]}]
correct_next_token = token_seq[{"seq": pz.slice[1:]}]


log_prob_of_correct_next = sliced_preds[{"vocabulary": correct_next_token}]
log_prob_of_correct_next

<NamedArray float32(| seq:55, batch:1) ≈-1.1e+01 ±0.29 [≥-1.2e+01, ≤-1e+01] nonzero:55 (wrapping jax.Array)
  <Arrayviz rendering>
| Device: GPU 0>

In [12]:
# Unbind the names, slice the positional array, rebind the names if needed.
sliced_preds = log_probs.untag("seq")[:-1].tag("seq")
correct_next_token = token_seq.untag("seq")[1:].tag("seq")
sliced_preds.untag("vocabulary")[correct_next_token]

<NamedArray float32(| seq:55, batch:1) ≈-1.1e+01 ±0.29 [≥-1.2e+01, ≤-1e+01] nonzero:55 (wrapping jax.Array)
  <Arrayviz rendering>
| Device: GPU 0>

# Visualize Attention Patterns in the Causal Self-Attention

In [ ]:
import penzai
from penzai import pz
from penzai.core.variables import StateVariable
from penzai.nn.layer import Layer
from typing import Any
from penzai.toolshed.unflaxify import InterceptedFlaxModuleMethod

@pz.pytree_dataclass  # <- This tags our class as being a Python dataclass and a JAX pytree node.
class DisplayIntermediateValue(pz.nn.Layer):  # <- pz.nn.Layer is the base class of Penzai layers.
  def __call__(self, intermediate_value, **unused_side_inputs):
    # print("Intermediate value:", intermediate_value)
    print(intermediate_value.shape)
    wrapped_intermediate_value = intermediate_value[0]
    # Show the value:
    pz.show(wrapped_intermediate_value)
    # And return it unchanged.
    return intermediate_value

def combined_call(original_submodule):
    # Create a wrapper that calls `original_submodule(...)`,
    # then does DisplayIntermediateValue or any post-processing.

    def new_callable(*args, **kwargs):
        output = original_submodule(*args, **kwargs)
        # do whatever side effect or instrumentation we want
        DisplayIntermediateValue()(output)  # e.g. just call it
        return output

    return new_callable

patched_selection = (
    pz.select(pz_model)
      .at_subtrees_where(
          lambda node:
              isinstance(node, InterceptedFlaxModuleMethod)
              and getattr(node.module, "__class__", None).__name__ == "Dropout"
              and node.module.name == 'attn_dropout'
      )
      
)


patched_model = patched_selection.apply(
          lambda method: combined_call(method),
      )

patched_selection


In [ ]:
repeated_sequence_prompt = "You are gentlemen of brave metal; you would lift the moon out of her sphere, if she would continue in it five weeks without changing."

# repeat the prompt twice
repeated_sequence_prompt = repeated_sequence_prompt + repeated_sequence_prompt

# Tokenize the prompt via BPE
tokens = tokenizer(repeated_sequence_prompt)

# Wrap tokens and tag with "batch" and "seq"
token_seq = pz.nx.wrap(tokens).tag("batch", "seq")

# Remove the tags and unwrap to get a plain JAX array
token_seq_plain = token_seq.untag("batch", "seq").unwrap()

# Now capture the plain array inside an ArgsAndKwargs instance.
token_seq_wrapped = ArgsAndKwargs.capture(token_seq_plain)


logits = patched_model(token_seq_wrapped)



(1, 4, 56, 56)


<jax.Array float32(4, 56, 56) ≈0.018 ±0.034 [≥0.0, ≤1.0] zero:6_160 nonzero:6_384
  <Arrayviz rendering>
| Device: GPU 0>

(1, 4, 56, 56)


<jax.Array float32(4, 56, 56) ≈0.018 ±0.034 [≥0.0, ≤1.0] zero:6_160 nonzero:6_384
  <Arrayviz rendering>
| Device: GPU 0>

(1, 4, 56, 56)


<jax.Array float32(4, 56, 56) ≈0.018 ±0.034 [≥0.0, ≤1.0] zero:6_160 nonzero:6_384
  <Arrayviz rendering>
| Device: GPU 0>

(1, 4, 56, 56)


<jax.Array float32(4, 56, 56) ≈0.018 ±0.034 [≥0.0, ≤1.0] zero:6_160 nonzero:6_384
  <Arrayviz rendering>
| Device: GPU 0>